In [ ]:
!wget https://huggingface.co/danjacobellis/dance/resolve/main/video_f8c48_encoder.pth

In [1]:
import torch
import datasets
import numpy as np
from autocodec.codec import AutoCodecND

In [2]:
ds = datasets.load_dataset("danjacobellis/davis3",split='validation').cast_column('video', datasets.Video()).with_format("torch")
checkpoint = torch.load('video_f8c48_encoder.pth', map_location="cpu",weights_only=False)
config = checkpoint['config']
state_dict = checkpoint['state_dict']
model = AutoCodecND(
    dim=3,
    input_channels=config.input_channels,
    J = int(np.log2(config.F)),
    latent_dim=config.latent_dim,
    encoder_depth = config.encoder_depth,
    encoder_kernel_size = config.encoder_kernel_size,
    decoder_depth = 1,
    lightweight_encode = config.lightweight_encode,
    lightweight_decode = True,
)
del model.decoder_blocks
model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
for sample in ds:
    video = sample['video']
    eight_frames = torch.cat([next(video)['data'].unsqueeze(1) for i_frame in range(8)],dim=1).unsqueeze(0)
    eight_frames = eight_frames[:,:,:,:240,:320]
    eight_frames = eight_frames.to(torch.float)/127.5 - 1.0
    break

In [4]:
eight_frames.shape

torch.Size([1, 3, 8, 240, 320])

In [5]:
%%time
with torch.no_grad():
    z = model.quantize.compand(model.encode(eight_frames)).round()

CPU times: user 5.54 s, sys: 444 ms, total: 5.99 s
Wall time: 1.65 s


In [6]:
8/1.65

4.848484848484849

In [7]:
for sample in ds:
    video = sample['video']
    eight_frames = torch.cat([next(video)['data'].unsqueeze(1) for i_frame in range(8)],dim=1).unsqueeze(0)
    eight_frames = eight_frames[:,:,:,:480,:640]
    eight_frames = eight_frames.to(torch.float)/127.5 - 1.0
    break

In [8]:
eight_frames.shape

torch.Size([1, 3, 8, 480, 640])

In [9]:
%%time
with torch.no_grad():
    z = model.quantize.compand(model.encode(eight_frames)).round()

CPU times: user 22.6 s, sys: 3 s, total: 25.6 s
Wall time: 7.26 s


In [10]:
8/7.26

1.1019283746556474